# Notebook for exploring claustrum data

### install and import essential packages and data

In [ ]:
#!pip install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

import scipy as sp
import scipy.io
import scipy.stats
import os
import numpy as np
import pandas as pd

from collections import Iterable
import matplotlib.pylab as mpl
%load_ext autoreload

mat = sp.io.loadmat('allUnitsLog_claus.mat')
mat2 = sp.io.loadmat('Opto_log_allUnits_claus.mat')
log = mat['fullLog']
log_df = pd.DataFrame(log, columns=['mouse_name', 'date', 'block_type', 'trial_type', 'touch_stimulus',\
                                    'vis_stimulus', 'response','trial_num', 'stim_onset', 'stim_offset', 'licks_right',\
                                    'licks_left', 'spike_times', 'cluster_name' ])
opto_log = mat2['fullOptoTable']
opto_log_df = pd.DataFrame(opto_log, columns=['mouse_name', 'date', 'cluster_name', 'opto_stim_onsets', 'opto_stim_offsets', 'spikes'])

In [ ]:
import h5py
f = h5py.File('waveform_log.mat')
f['waveform_log']

data = [f[element[0]][:] for element in f['waveform_log']]
    
data[3]

In [ ]:
mat3 = sp.io.loadmat('waveform_log.mat')

In [ ]:
waveforms = mat3['waveform_log']
waveforms_df = pd.DataFrame(waveforms, columns = ['mouse_name', 'date', 'tetrode_num', 'spikes', 'waveforms'])

waveforms_df.head(3)

### clean up data tables

In [ ]:

for col in [0,1,2,3,4,5,6,6,7,7,7,7,8,8,8,8,9,9,9,9,10,10,11,11,13]:
        log_df.ix[:,col] = log_df.ix[:,col].str[0]
for col in [0,1,2,3,4]:
        opto_log_df.ix[:,col] = opto_log_df.ix[:,col].str[0]
for col in [0,1,2]:
    waveforms_df.ix[:,col] = waveforms_df.ix[:,col].str[0]
    


In [ ]:
waveforms_df['spikes']= waveforms_df['spikes'].apply(lambda y: np.concatenate(y) if len(y) > 0 else y)
opto_log_df['spikes'] = opto_log_df['spikes'].apply(lambda y: np.concatenate(y) if len(y) > 0 else y)

In [ ]:
log_df.head(2)

In [ ]:
opto_log_df.head(2)

In [ ]:
waveforms_df.head(5)

In [ ]:
clusters = opto_log_df['cluster_name'].copy()
clusters = clusters.apply(lambda y: y[:3])

opto_log_df['waveforms'] = np.nan
opto_log_df['waveforms'] = opto_log_df['waveforms'].astype(object)
for unit in range(len(opto_log_df)):
    tetrode_idx = pd.concat((opto_log_df.loc[unit,['mouse_name', 'date']] == waveforms_df[['mouse_name', 'date']],
           clusters[unit] == waveforms_df['tetrode_num']), axis = 1).all(axis = 1)
    tetrode = waveforms_df[tetrode_idx]
    
    spike_idx = np.in1d(np.around(tetrode['spikes'].iloc[0],4),  np.around(opto_log_df.loc[unit, 'spikes'],4))
    wv = tetrode['waveforms'].iloc[0][spike_idx]
    opto_log_df.loc[unit,'waveforms'] = wv

In [ ]:
opto_log_df['waveforms1'] = np.nan
opto_log_df['waveforms2'] = np.nan
opto_log_df['waveforms3'] = np.nan
opto_log_df['waveforms4'] = np.nan

opto_log_df['waveforms1'] = opto_log_df['waveforms'].apply(lambda y: y[:,0,:])
opto_log_df['waveforms2'] = opto_log_df['waveforms'].apply(lambda y: y[:,1,:])
opto_log_df['waveforms3'] = opto_log_df['waveforms'].apply(lambda y: y[:,2,:])
opto_log_df['waveforms4'] = opto_log_df['waveforms'].apply(lambda y: y[:,3,:])
del opto_log_df['waveforms']

### sort optogenetic stimulus pulses into groups based on frequency of stimulation

In [ ]:
unique_sessions = opto_log_df[['mouse_name', 'date']].drop_duplicates()

opto_log_df['first_last_opto_pulses'] = np.nan
opto_log_df['grouped_opto_pulses'] = np.nan

for session in range(unique_sessions.shape[0]):
    session_row_ind = (opto_log_df.loc[:,['mouse_name', 'date']] == unique_sessions.iloc[session]).all(axis=1)
    rows = opto_log_df.loc[session_row_ind]
    
    ISIs = np.around(rows.iloc[0,3][1:]-rows.iloc[0,3][0:-1], 4)
    ISIs = np.concatenate(([ISIs[0]], ISIs, [ISIs[-1]]))
    unique_ISIs = np.unique(ISIs, return_counts = True)
    ind = unique_ISIs[1]>50
    unique_ISIs = unique_ISIs[0][ind]

    first_last_opto_pulses = {}
    grouped_opto_pulses = {}
    for isi in unique_ISIs:
        ISI_category = np.where(np.absolute(ISIs-isi) <= 0.001)[0]
        opto_pulse_inds= np.array(range(np.min(ISI_category),np.max(ISI_category)))
        grouped_opto_pulses[1/isi] = rows.iloc[0,3][opto_pulse_inds]

        IBI_ind = ISIs[opto_pulse_inds] != isi
        first_pulse_ind = np.concatenate(([opto_pulse_inds[0]],opto_pulse_inds[IBI_ind]))
        last_pulse_ind = np.concatenate((first_pulse_ind[1:]-1, [opto_pulse_inds[-1]]))
        first_last_opto_pulses[1/isi] = [rows.iloc[0,3][first_pulse_ind],rows.iloc[0,3][last_pulse_ind]]
    opto_log_df.loc[session_row_ind,'first_last_opto_pulses'] = [first_last_opto_pulses]
    opto_log_df.loc[session_row_ind,'grouped_opto_pulses'] = [grouped_opto_pulses]

In [ ]:
opto_log_df.head(3)

### plot spike rasters for optogenetic stimulus trials
#### define function that will plot rasters

In [ ]:
import matplotlib.patches as patches
from matplotlib import gridspec
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 18}

mpl.rc('font', **font)
mpl.rc('xtick', labelsize=16) 
mpl.rc('ytick', labelsize=16)
mpl.rc('axes', labelsize=18)

    

def opto_identify(unit_num, log):
    
    unit = opto_log_df.iloc[unit_num]
    mpl.close('all')
    fig = mpl.figure(figsize=(15, 8))
    fig.suptitle(unit['mouse_name'] + ', ' +  unit['date'] + ', ' + unit['cluster_name'])
    gs = gridspec.GridSpec(5, 2, height_ratios=[1, 15, 5, 1, 15]) 
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])
    ax3 = fig.add_subplot(gs[6])
    ax4 = fig.add_subplot(gs[7])

    rs1 = fig.add_subplot(gs[2])
    rs2 = fig.add_subplot(gs[3])
    rs3 = fig.add_subplot(gs[8])
    rs4 = fig.add_subplot(gs[9])

    rasters = [rs1, rs2, rs3, rs4]
    opto_axes = [ax1, ax2, ax3, ax4]
    frequencies = [1,5,10,40]

    for i, frequency in enumerate(frequencies):
        flop = unit['first_last_opto_pulses'][frequency]

        trial_dur = flop[1][0] - flop[0][0]
        trial_total = 0
        for trial in range(len(flop[0])):
            trial_spike_inds = (flop[0][trial]-0.5 < unit['spikes']) & (unit['spikes'] < flop[1][trial]+.5)
            trial_spikes = unit['spikes'][trial_spike_inds] - flop[0][trial]
            rasters[i].vlines(trial_spikes, trial + .5, trial + 1.3, linewidth = 0.5)
            trial_total += 1

        figure_pulse_inds = [(unit['grouped_opto_pulses'][frequency] >= flop[0][1])
                  & (unit['grouped_opto_pulses'][frequency] <= flop[1][1])]
        figure_pulses = unit['grouped_opto_pulses'][frequency][figure_pulse_inds] - flop[0][1]
        example_pulse_ind = np.where(unit['opto_stim_onsets'] == flop[0][1])
        stim_duration = unit['opto_stim_offsets'][example_pulse_ind] - unit['opto_stim_onsets'][example_pulse_ind]

        for p in figure_pulses:
            opto_axes[i].add_patch(patches.Rectangle((p,0), stim_duration*5, 4, color = 'xkcd:sky blue')) 

        rasters[i].autoscale(enable=True, tight=True)
        rasters[i].spines['right'].set_visible(False)
        rasters[i].spines['top'].set_visible(False)
        rasters[i].xaxis.set_ticks_position('bottom')
        rasters[i].yaxis.set_ticks_position('left')
        rasters[i].set_xlim(-trial_dur*0.1, trial_dur + trial_dur*0.1)
        rasters[i].set_xlabel('Time(s)')
        rasters[i].set_ylabel('Trials')

        opto_axes[i].set_ylim(0, 4, )
        opto_axes[i].set_xlim(-trial_dur*0.1, trial_dur + trial_dur*0.1 )
        opto_axes[i].axis('off')
        opto_axes[i].set_title(str(frequency) + ' Hz')

    mpl.subplots_adjust(left=0.1, right=.9, top=0.9, bottom=0.1)

    return fig

#### create widget that will make exploring data easier - drag slider to switch between units

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.html import widgets
from IPython.display import display
import traitlets

log = opto_log_df
unit_num = 12
num = widgets.IntSlider(min = 1, max = opto_log_df.shape[0], step = 1, width='75%', height='30px', 
                    description= "Unit", continuous_update=False)
s = interactive(opto_identify, unit_num = num, log = fixed(log), continuous_update=False)
display(s)

### plot reliability and latency of optogenetic responses
#### calculate identification metrics for each unit

In [ ]:
all_opto_metrics_df = None
for row in range(len(opto_log_df)):
    unit = opto_log_df.iloc[row]

    first_pulse = np.min(np.concatenate(list(unit['grouped_opto_pulses'].values())))
    last_pulse = np.max(np.concatenate(list(unit['grouped_opto_pulses'].values())))
    cont_spike_inds = (unit['spikes'] < first_pulse) | (unit['spikes'] > last_pulse)

    frequencies = [1,5,10,40]

    opto_windows_df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in unit['grouped_opto_pulses'].items()]))
    del opto_windows_df[2]
    names = [str(freq) + 'end' for freq in frequencies]
    evoked_spike_inds_df = opto_windows_df.applymap(lambda y: np.where((y <= unit['spikes']) & (y+0.01>= unit['spikes']))[0]) 

    opto_metrics_df= pd.DataFrame(frequencies, columns = ['frequency'])
    opto_metrics_df['unit_number'] = row
    opto_metrics_df['mouse_name'] = unit['mouse_name']
    opto_metrics_df['date'] = unit['date']
    opto_metrics_df['cluster_name'] = unit['cluster_name']

    spike_inds = []
    for freq in frequencies:
        spike_inds.append(np.concatenate(evoked_spike_inds_df[freq].as_matrix()))
    opto_metrics_df['spike_inds'] = pd.Series(spike_inds)
    pulses = pd.DataFrame([[unit['grouped_opto_pulses'][freq]] for freq in frequencies],
                          columns = ['pulses'])
    opto_metrics_df['pulses'] = pulses

    def find_latencies(s):
        spikes = unit['spikes']
        idx = spikes.searchsorted(s['pulses'], side = 'right')
        idx2 = idx[idx < len(unit['spikes'])] #otherwise will return an error if no spikes follow the last laser pulse
        raw_latencies = spikes[idx2] - s['pulses'][0:len(idx2)]
        if len(raw_latencies <= 0.01) > 0:
            reliability = sum(raw_latencies <= 0.01)/len(raw_latencies <= 0.01)
        else:
            reliability = 0
        latencies = raw_latencies[raw_latencies <= 0.01]
        return [[latencies], reliability]
    # raw_latencies, latencies
    opto_metrics_df['reliability'] = opto_metrics_df.apply(lambda y: find_latencies(y)[1], axis = 1)
    opto_metrics_df['latencies'] = opto_metrics_df.apply(lambda y: find_latencies(y)[0], axis = 1)
    opto_metrics_df['latencies'] = opto_metrics_df['latencies'].apply(lambda y: y[0])
    opto_metrics_df['mean_latencies'] = opto_metrics_df['latencies'].apply(lambda y: np.mean(y) if len(y)>0 else None)
    
    waveform_corr = []
    for waveform in ['waveforms1','waveforms2','waveforms3','waveforms4']:    
        opto_metrics_df[waveform] = opto_metrics_df['spike_inds'].apply(
            lambda y: np.mean(unit[waveform][y], axis=0))
        opto_metrics_df['cont_'+waveform] = [np.mean(unit[waveform][cont_spike_inds], axis=0)]*4
        waveform_corr.append([sp.stats.pearsonr(opto_metrics_df[waveform][i],
                           opto_metrics_df['cont_'+waveform][i])[0] for i in range(4)])
    opto_metrics_df['waveform_corr'] = np.around(np.mean(waveform_corr),2)

    
    if all_opto_metrics_df is None: 
        all_opto_metrics_df = opto_metrics_df
    else:
        all_opto_metrics_df = all_opto_metrics_df.append(opto_metrics_df)
all_opto_metrics_df = all_opto_metrics_df.reset_index(drop = True)

In [ ]:
all_opto_metrics_df.head(3)

#### define function that will plot identification metrics for each unit

In [ ]:
import seaborn as sns
sns.set_style("ticks")

def opto_metrics(unit_num, log, opto_metrics_log):

    unit = all_opto_metrics_df[all_opto_metrics_df['unit_number'] == unit_num]
    sub_log_df = opto_log_df.iloc[unit_num]
    frequencies = [1,5,10,40]
    stim_duration = 0.002

    mpl.close('all')
    fig2 = mpl.figure(figsize=(18, 10))
    fig2.suptitle(sub_log_df['mouse_name'] + ', ' +  sub_log_df['date'] + ', ' + sub_log_df['cluster_name'])

    ax1 = mpl.subplot2grid((2,2), (0,0), rowspan=2, colspan=1)
    ax2 = mpl.subplot2grid((2,2), (0,1), rowspan=1, colspan=1)
    ax3 = mpl.subplot2grid((2,2), (1,1), rowspan=1, colspan=1)

    ax4 = fig2.add_axes([0.34, 0.425, 0.1, 0.4])
    sns.despine()

    data={}
    total_stims = 0

    laser_evoked_spike_inds = []
    for freq in frequencies:
        opto_stims = sub_log_df['grouped_opto_pulses'][freq]
        for stim_num in range(90):
            spike_inds = (sub_log_df['spikes'] >= opto_stims[stim_num]-0.02) & (sub_log_df['spikes'] 
                                                                                < opto_stims[stim_num] + .03)
            spikes = (sub_log_df['spikes'][spike_inds] - opto_stims[stim_num])*1000
            ax1.vlines(spikes, total_stims + stim_num + .5, total_stims + stim_num + 1.3, linewidth = 2)
            x = total_stims+stim_num

        ax1.plot([0,stim_duration*1000], [total_stims+stim_num, total_stims+stim_num], color = 'k', linewidth = 3)
        ax1.text(-5, total_stims + stim_num/2, str(freq) + ' Hz', fontsize = 14),
        total_stims = total_stims + stim_num

    frequencies = unit['frequency']
    values = np.concatenate(list(unit['latencies']))
    labels = np.concatenate([[freq] * len(unit['latencies'].iloc[i])
                             for i, freq in enumerate(frequencies)])
    data_df = pd.DataFrame([values*1000, labels], index = ['Latency (ms)', 'Frequency (Hz)']).T
    sns.violinplot(x = 'Frequency (Hz)', y = 'Latency (ms)', data = data_df, ax=ax3, color = 'xkcd:sky blue')

    sns.pointplot(x = 'frequency', y = 'reliability', data = unit, ax=ax2, join = False, color = 'xkcd:sky blue')

    offset = 0
    waveform_corr = []
    for electrode in ['waveforms1','waveforms2','waveforms3','waveforms4']:
        laser_evoked_waveforms_df = pd.DataFrame(np.mean(unit[electrode]))+offset
        cont_waveforms_df = pd.DataFrame(np.mean(unit['cont_'+electrode]))+offset
        ax4.plot(cont_waveforms_df, color = 'k', alpha = 0.8, linewidth = 3)
        ax4.plot(laser_evoked_waveforms_df, color = 'xkcd:sky blue',alpha = 0.8, linewidth = 3)
        offset += 0.2
    ax4.set_title('r = ' + str(unit['waveform_corr'].iloc[0]), fontsize = 14)

    ax1.set_xlabel('Time(ms)')
    ax1.set_ylabel('Stim trial')
    ax1.autoscale(enable=True, tight=True)
    ax1.xaxis.set_ticks_position('bottom')
    ax1.yaxis.set_ticks_position('left')
    ax1.add_patch(patches.Rectangle((0,0), stim_duration*1000, total_stims, color = 'xkcd:sky blue'))
    ax1.set_xlim(-20,30)

    ax3.set_xlabel('Frequency (Hz)')
    ax3.set_ylabel('Mean spike latency (ms)')
    ax3.set_ylim(0,20)
    ax3.yaxis.set_ticks(np.arange(0, 20, 2.5))

    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('P(spike)')
    ax2.set_ylim(0,1)

    ax4.patch.set_alpha(0)
    ax4.axis('off')

    ax5 = fig2.add_axes([0.40, 0.40, 0.05, 0.1])
    ax5.spines['left'].set_visible(False)
    ax5.spines['top'].set_visible(False)
    ax5.yaxis.set_ticks_position('right')
    ax5.spines['right'].set_smart_bounds(True)
    ax5.spines['bottom'].set_smart_bounds(True)
    mpl.yticks((0,0.2,0.2), fontsize = 13)
    mpl.xticks((0,0.5,0.5), fontsize = 13)
    ax5.set_ylim(0,0.2)
    ax5.set_xlim(0,0.5)
    ax5.patch.set_alpha(0)
    ax5.set_xlabel('ms',fontsize = 13)
    ax5.set_ylabel('mV',fontsize = 13)
    ax5.yaxis.set_label_position('right')

    return fig2

#### create widgets to make exploring data easier

In [ ]:
num = widgets.IntSlider(min = 1, max = opto_log_df.shape[0], step = 1, width='75%', height='30px', 
                    description= "Unit", continuous_update=False)
s = interactive(opto_metrics, unit_num = num, log = fixed(log), opto_metrics_log = fixed(all_opto_metrics_df), continuous_update=False)
display(s)

In [ ]:
mpl.savefig('Cl3_03-27-17_TT3c2(highReliability).png',format='png', dpi=900)

In [ ]:
import colorlover as cl
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('efinkel11', 'yhtY1t0dIUH4hX37eXdO')
traces = []

colors = ['rgb(236,231,242)','rgb(166,189,219)','rgb(43,140,190)', 'rgb(10,50,110)']
frequencies = [1,5,10,40]
for i, freq in enumerate(frequencies):
    rows = all_opto_metrics_df['frequency'] == freq
    x = all_opto_metrics_df['mean_latencies'][rows]
    y = all_opto_metrics_df['waveform_corr'][rows]
    z = all_opto_metrics_df['reliability'][rows]
    n = all_opto_metrics_df.loc[rows, ['mouse_name',
                                'date', 'cluster_name']].as_matrix()
    clust_names = [' '.join(name) for name in n]
    clust_names = ['.'.join((ns[0:2]+ns[9],ns[12],ns[14:16],ns[17:])) for ns in clust_names]

    trace = go.Scatter3d(
        text = clust_names,
        x=x, y=y, z=z,
        mode = 'markers',
        marker=dict(
            size=6,
            opacity = 0,
            color = colors[i]),
        line = dict(
            width = 0,
            color = colors[i]),
        name = str(freq)+'Hz',)
    traces.append(trace)
data = traces

layout = dict(
    width=1000,
    height=800,
    autosize=False,
    legend = dict(orientation= "h"),
    scene=dict(
        xaxis=dict(title = 'mean spike latency'),
        yaxis=dict(title = 'waveform corr'),
        zaxis=dict(
            title = 'p(spike)')))

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='pandas-brownian-motion-3d', height=700, validate=False)

In [ ]:
log_df['stim_onset'] = log_df['stim_onset'].fillna(0)
log_df['spike_times'] = log_df['spike_times'] - log_df['stim_onset']
licks = pd.concat([log_df['licks_right'] - log_df['stim_onset'] , log_df['licks_left']-log_df['stim_onset']], axis=1)
licks = licks.applymap(lambda y: y[[0.1<y]] if len(y) > 0 else y)
licks = licks.applymap(lambda y: y[[1>y]] if len(y) > 0 else y)
licks = licks.applymap(lambda y: min(y) if len(y) > 0 else np.nan)
log_df['first_lick'] = licks.min(axis=1)

log_df[['licks_right','licks_left', 'spike_times']] = \
log_df[['licks_right','licks_left', 'spike_times']].applymap(lambda x: np.concatenate(x) if len(list(x)) > 0 else x)

log_df = log_df.sort_values(['mouse_name', 'date', 'cluster_name', 'first_lick'], ascending = [1,1,1,1])
log_df['identified'] = 'unidentified'
log_df

In [ ]:
subset_dict = {}
size_dict = {}
subset_dict['None'] = 0
size_dict['None'] = 0

categories = np.concatenate([log_df['mouse_name'].unique(), log_df['identified'].unique()])

for cat in categories:
        subset = log_df[log_df['mouse_name'] == cat]
        if subset.size == 0:
            subset = log_df[log_df['identified'] == cat]
            print(cat)
        subset_dict[cat] = subset
        unique_units = subset[['mouse_name', 'date', 'cluster_name']].drop_duplicates()
        size_dict[cat] = len(unique_units)
        #print(unique_units.size)

In [ ]:
%autoreload 2
%timeit
import ind_unit as iu
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.html import widgets
from IPython.display import display
import traitlets


w = widgets.Dropdown(options = list(subset_dict.keys()), value = 'None', description='cellType:')

num = widgets.IntSlider(min = 1, max = 50, step = 1, width='75%', height='30px', 
                    description= "Unit", continuous_update=False)
x1 = widgets.FloatSlider(min = -.9, max = 0, step = 0.05, value = -0.25,width='75%',
                          height='30px', description= "xMin", continuous_update=False)
x2 = widgets.FloatSlider(min = 0, max = 2, step = 0.05, value=0.75,width='75%', height='30px',
                          description= "xMax",continuous_update=False)
s = interactive(iu.plot_unit, df_dict = fixed(subset_dict), s_key = w, n=num, x_min = x1, x_max = x2, continuous_update=False)


def update_max(*args):
    num.value = 1
    num.max = size_dict[w.value]
w.observe(update_max, 'value')

display(s)

In [ ]:
mpl.savefig('Cl3_03-15-17_TT6c4_rasters.png',format='png', dpi=900)

In [ ]:
ar

In [ ]:
len(ar)